# MATCHING MARKETS


Some definitions, theorems, and algorithms:

`# TODO(sparshsah): i kinda just nested the defns into the thms lol`

* <strong>Thm0 (Existence of a Perfect Matching)</strong> :
    * A bipartite network $G$
        * (a network whose nodes are separable into two subsets $U$ and $V$
        * such that the neighborhood $N(u)$ of any node $u \in U$
        * does not include any other node $u' \in U$,
        * and similarly for $V$)
    * with $|U| = |V|$
    * has a perfect matching
    * iff it does not contain a set of constricted nodes
        * (a subset of nodes $U_C \subseteq U$
        * such that the neighborhood $N(U_C) = \bigcup_{u \in U_C} N(u)$ of $U_R$
        * is smaller than the number of nodes $|U_C|$---
        * or similarly for $V$).


* <strong>Thm1a (Existence of a Set of Socially-Optimal Market-Clearing Prices)</strong> :
    * For any set of buyer valuations (nonnegative real numbers) for each seller,
    * there exists a set of market-clearing seller prices (nonnegative real numbers)
    * that produces a socially-optimal outcome, i.e.
        * the induced perfect matching in the resulting preferred-seller network
            * (a bipartite subnetwork of the main network $G$
            * such that each buyer is linked to at most one seller --
            * either a single seller who maximizes the payoff to that buyer, with ties broken arbitrarily
            * or, if no seller can provide a nonnegative payoff, then no seller at all)
        * maximizes the sum of payoffs to all buyers and sellers.


* <strong>Thm1b (Social Optimality of Any Set of Market-Clearing Prices)</strong> :
    * For any set of market-clearing prices,
    * any perfect matching in the resulting preferred-seller network
    * is socially-optimal.

In [61]:
from typing import List
# data structures
import pandas as pd

# each row is a buyer,
# each column is that buyer's valuation for corresponding seller
BuyerVals = pd.DataFrame
def _get_buyer_name(b: int) -> str:
    return f"B{b}"
def _make_buyer_vals(buyer_vals: List[list]) -> BuyerVals:
    buyer_vals = BuyerVals(buyer_vals, dtype=float)
    buyer_vals = buyer_vals.rename(
        index=_get_buyer_name, columns=_get_seller_name
    )
    return buyer_vals

SellerPrices = pd.Series
def _get_seller_name(s: int) -> str:
    return f"S{s}"
def _make_seller_prices(num_sellers: int) -> SellerPrices:
    seller_prices = SellerPrices(0, index=range(num_sellers), dtype=float)
    seller_prices = seller_prices.rename(index=_get_seller_name)
    return seller_prices

# arbitrarily, rows are buyers and columns are sellers
# this is dtype=bool
Links = pd.DataFrame
def _make_links(index, columns) -> Links:
    return Links(False, index=index, columns=columns, dtype=bool)


class _BuyerSellerNetwork:
    """A bipartite network with `|Buyers| == |Sellers|`."""
    def __init__(self, buyer_vals: BuyerVals, seller_prices: SellerPrices):
        self.buyer_vals = BuyerVals(buyer_vals, dtype=float)
        self.seller_prices = SellerPrices(seller_prices, dtype=float)
        self._validate()
    
    def _validate(self) -> bool:
        # the number of buyers is same as number of sellers
        assert len(buyer_vals.index) == len(seller_prices.index), \
            f"There are {len(self.seller_prices.index)} sellers, but " + \
            f"{len(self.buyer_vals.index)} buyers!"
        # each buyer assigns exactly one valuation to each seller
        assert len(buyer_vals.columns) == len(seller_prices.index), \
            f"There are {len(self.seller_prices.index)} sellers, but " + \
            f"buyer valuations exist for {len(self.buyer_vals.columns)}!"
        # each buyer must set a valuation for every seller
        assert not self.buyer_vals.isnull().any().any(), \
            f"Buyer valuations\n{self.buyer_vals}\ncontain NaN's!"
        return True
    
    def set_sellers_price(self, seller: int, price: float) -> bool:
        self.seller_prices.loc[seller] = price
        return True
    
    def __repr__(self):
        head = f"Buyer<-Seller valuations:\n{self.buyer_vals}"
        tail = f"Seller prices:\n{self.seller_prices}"
        _repr = f"{head}\n\n{tail}"
        return _repr
            

class _PricedBuyerSellerNetwork(_BuyerSellerNetwork):
    def _validate(self) -> bool:
        super()._validate()
        # seller prices can't be None here
        assert not self.seller_prices.isnull().any(), \
            f"Seller prices\n{self.seller_prices}\ncontain NaN's!"
        return True

class BuyerSellerNetwork(_PricedBuyerSellerNetwork):
    def __init__(
            self, buyer_vals: BuyerVals, seller_prices: SellerPrices,
            links: Links=None
        ):
        super().__init__(buyer_vals=buyer_vals, seller_prices=seller_prices)
        if links is None:
            links = _make_links(
                index=buyer_vals.index,
                columns=seller_prices.index
            )
        self.links = links
        
    def set_link(buyer: int, seller: int) -> bool:
        self.links.loc[buyer, seller] = True
        return True
    
    def __repr__(self):
        head = super().__repr__()
        tail = f"Buyer->Seller links:\n{self.links}"
        _repr = f"{head}\n\n{tail}"
        return _repr


buyer_vals = _make_buyer_vals([
    [12, 4, 2],
    [8, 7, 6],
    [7, 5, 2]
])
seller_prices = _make_seller_prices(num_sellers=len(buyer_vals.index))
g = BuyerSellerNetwork(buyer_vals=buyer_vals, seller_prices=seller_prices)
g

Buyer<-Seller valuations:
      S0   S1   S2
B0  12.0  4.0  2.0
B1   8.0  7.0  6.0
B2   7.0  5.0  2.0

Seller prices:
S0    0.0
S1    0.0
S2    0.0
dtype: float64

Buyer->Seller links:
       S0     S1     S2
B0  False  False  False
B1  False  False  False
B2  False  False  False

# Algorithms

## Find maximal matching

We'll use the augmenting-path algorithm. Given a (possibly sub-maximal) matching, an alternating path w.r.t. that matching is a path that alternates between links that are not in and links that are in the matching. An augmenting path w.r.t. that matching is an alternating path with unmatched endpoints. Once we've found an augmenting path, we can find a larger matching by toggling whether the links are in the matching. This works because (fencepost principle) there is always exactly one more non-matching link than matching links in the augmenting path.

In [58]:
def is_matching(links: Links) -> bool:
    """Whether `links` correctly encodes a matching."""
    links_from_lhs_to_rhs = links.sum(axis="columns")
    if links_from_lhs_to_rhs.max() > 1:
        return False
    del links_from_lhs_to_rhs
    links_from_rhs_to_lhs = links.sum(axis="index")
    if links_from_rhs_to_lhs.max() > 1:
        return False
    del links_from_rhs_to_lhs
    return True


def _get_unmatched_node(links: Links, rhs: bool=False) -> int:
    """Get some unmatched node. Arbitrarily, returns first one."""
    if rhs:
        links = links.T  # :)
    # whether the node is unmatched
    is_unmatched = links.sum(axis="columns") < 1
    unmatched_nodes = links[is_unmatched]
    return unmatched_nodes.iloc[0]


def _get_neighborhood(
        links: Links, node: int=0,
        complement: bool=False, rhs: bool=False
    ):
    if complement:
        links = ~links
    if rhs:
        links = links.T
    # bool, whether the link is set
    links_of_node = links.loc[node, :]
    neighborhood = links.columns[links_of_node]
    neighborhood = list(neighborhood)
    return neighborhood


def augment(matching: Links) -> Links:
    """Assumes `links` represents a buyer-seller network."""
    matching = matching.copy()
    node = _get_unmatched_node(links=matching)
    # whether the current node is on the RHS
    curr_rhs = False
    # whether the current node is in the matching
    curr_match = False
    # we perform a variation of BFS
    nodes = [node,]
    while len(nodes):
        # keep this straight:
        # if the curr node is matched,
        # we want to FLIP and traverse links NOT IN the matching.
        neighbors = _get_neighborhood(
            links=links, node=node,
            complement=curr_match, rhs=rhs
        )

In [62]:
links = _make_links(index=buyer_vals.index, columns=seller_prices.index)
links

,S0,S1,S2
B0,False,False,False
B1,False,False,False
B2,False,False,False


In [68]:
~links

,S0,S1,S2
B0,True,True,True
B1,True,True,True
B2,True,True,True


In [66]:
links.columns[~links.loc["B0", :]]

Index(['S0', 'S1', 'S2'], dtype='object')

## Auction